In [2]:
# --- Project Setup ---
from setup_notebook import setup_project_root
setup_project_root()

# --- Imports ---
from src.project_config import AM_PATH, ESM_PATH, PROTEIN_IDS_CSV, get_paths_protein
import pandas as pd


## Clustering Algorithm for identifying clusters of high difference between the two models AlphaMissense and ESM1b

In [6]:
paths = get_paths_protein("P05067")

#diff_matrix = pd.read_csv(paths["difference_path"], index_col=0)

diff_matrix_new = pd.read_csv(paths["difference_path"], index_col=0)

avg_diff_per_position = diff_matrix.mean(axis=0)  # shape: (N,)
avg_diff_per_position_new = diff_matrix_new.mean(axis=0)  # shape: (N,) 
print(avg_diff_per_position)
print(avg_diff_per_position_new)



M 1      0.291113
L 2      0.215340
P 3      0.157512
G 4      0.105954
L 5      0.182534
           ...   
E 766    0.000096
Q 767    0.168294
M 768    0.098841
Q 769    0.269723
N 770    0.177455
Length: 770, dtype: float64
M 1      0.291141
L 2      0.214885
P 3      0.157159
G 4      0.105609
L 5      0.182185
           ...   
E 766    0.000301
Q 767    0.168859
M 768    0.099376
Q 769    0.270177
N 770    0.178020
Length: 770, dtype: float64


In [5]:
paths = get_paths_protein("P05067")
avg_diff = pd.read_csv(paths["difference_path"], index_col=0).mean(axis=0)


avg_diff

M 1      0.291141
L 2      0.214885
P 3      0.157159
G 4      0.105609
L 5      0.182185
           ...   
E 766    0.000301
Q 767    0.168859
M 768    0.099376
Q 769    0.270177
N 770    0.178020
Length: 770, dtype: float64

In [ ]:
window_avg = avg_diff_per_position[i : i + window_size].mean()
